In [2]:
from src.extract_frames import *

In [3]:
video_list = ["dyson1.mp4", "dyson2.mp4", "dyson3.mp4"]
store_frames(video_list, "video/", "frame/")

Reading file: video/dyson1.mp4
Reading file: video/dyson2.mp4
Reading file: video/dyson3.mp4


In [28]:
from keras.applications import ResNet152
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img

import progressbar
import h5py
import numpy as np

In [31]:
# Image Size to input the model
IMG_SIZE=(224, 224)
# frame numbers
FRAME_NUMBERS = [1, 25, 50, 75, 100, 125, 150, 175]

def get_features_from_last_layer_pretrained_nn(videos, frames_path):
    model = ResNet152(weights="imagenet", include_top=False, pooling='avg')
    size = 2048
    h5_filename = '{}/{}.h5'.format("feature", "ResNet152")

    if not os.path.isfile(h5_filename):

        print('[INFO] Creating features for pre-trained model...')

        # images
        image_paths = [
            os.path.join(frames_path, video.split('.webm')[0] + '-frame-{}.jpg'.format(frame))
            for video in videos
            for frame in FRAME_NUMBERS
        ]

        # initialize the progress bar
        widgets = ["Extracting Features: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
        pbar = progressbar.ProgressBar(maxval=len(image_paths), widgets=widgets).start()

        batch_size = 32

        with h5py.File(h5_filename, 'w') as h5f:

            # loop over the images in batches
            for i in np.arange(0, len(image_paths), batch_size):

                batch_paths = image_paths[i:i + batch_size]
                batch_images = []

                # loop over the images in the current batch
                for image_path in batch_paths:

                    # load the input image using the Keras helper utility
                    # while ensuring the image is resized to 224x224 pixels
                    image = load_img(image_path, target_size=IMG_SIZE)
                    image = img_to_array(image)
                    # preprocess the image by (1) expanding the dimensions and
                    # (2) subtracting the mean RGB pixel intensity from the
                    # ImageNet dataset
                    image = np.expand_dims(image, axis=0)
                    image = imagenet_utils.preprocess_input(image)
                    batch_images.append(image)

                # pass the images through the network and use the outputs as
                # our actual features
                batch_images = np.vstack(batch_images)
                features = model.predict(batch_images, batch_size=batch_size)
                # reshape the features so that each image is represented by
                # a flattened feature vector of the ‘MaxPooling2D‘ outputs
                features = features.reshape((features.shape[0], size))
                print(features.shape)

                # add features per image
                for feature_index, batch_index in zip(np.arange(0, features.shape[0], len(FRAME_NUMBERS)),
                                                    np.arange(i, i + len(FRAME_NUMBERS))):
                    # Concatenate all features per frame to one
                    image_features = np.concatenate( features[feature_index:feature_index + len(FRAME_NUMBERS)] )
                    # grab the video corresponding to these frames
                    image_index = batch_index // len(FRAME_NUMBERS) + batch_index % len(FRAME_NUMBERS)
                    video_name = videos[image_index]
                    print(video_name)
                    # save it!
                    h5f.create_dataset(video_name, data=image_features)

                # update the progress bar
                pbar.update(i)

        # progress bar
        pbar.finish()

    # Reading file

    print('[INFO] Reading features from {}...'.format(h5_filename))

    with h5py.File(h5_filename) as h5f:

        features = np.array([ h5f[video_name][:] for video_name in videos ])

    return features

In [32]:
features = get_features_from_last_layer_pretrained_nn(video_list, "frame/")

Extracting Features: N/A% |                                    | ETA:  --:--:--

[INFO] Creating features for pre-trained model...


Extracting Features: 100% |####################################| Time:  0:00:05


(24, 2048)
dyson1.mp4
dyson2.mp4
dyson3.mp4
[INFO] Reading features from feature/ResNet152.h5...


/Users/jojo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:81: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


In [36]:
features.shape

(3, 16384)